In [1]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader

import lime

import matplotlib.pyplot as plt
import os 
import sys
import random
import warnings

import torch

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# df2023 = pd.read_csv("./data/waqi-covid-years/waqi-covid-2023.csv")
# print(df2023['Country'].value_counts())
# print(df2023['Country'].nunique())
# print(df2023['Country'].unique())

In [3]:
# Load Data
Countries = ['US','CN','JP','ES','IN','KR','FR','GB','TR','DE','RO','CA','IR']
Country = Countries[12]

# df2019Q1 = pd.read_csv("./data/waqi-covid-years/waqi-covid-2019Q1.csv")
# df2019Q2 = pd.read_csv("./data/waqi-covid-years/waqi-covid-2019Q2.csv")
# df2019Q3 = pd.read_csv("./data/waqi-covid-years/waqi-covid-2019Q3.csv")
# df2019Q4 = pd.read_csv("./data/waqi-covid-years/waqi-covid-2019Q4.csv")

# df2020Q1 = pd.read_csv("./data/waqi-covid-years/waqi-covid-2020Q1.csv")
# df2020Q2 = pd.read_csv("./data/waqi-covid-years/waqi-covid-2020Q2.csv")
# df2020Q3 = pd.read_csv("./data/waqi-covid-years/waqi-covid-2020Q3.csv")
# df2020Q4 = pd.read_csv("./data/waqi-covid-years/waqi-covid-2020Q4.csv")

df2021Q1 = pd.read_csv("./data/waqi-covid-years/waqi-covid-2021Q1.csv")
df2021Q2 = pd.read_csv("./data/waqi-covid-years/waqi-covid-2021Q2.csv")
df2021Q3 = pd.read_csv("./data/waqi-covid-years/waqi-covid-2021Q3.csv")
df2021Q4 = pd.read_csv("./data/waqi-covid-years/waqi-covid-2021Q4.csv")

df2023 = pd.read_csv("./data/waqi-covid-years/waqi-covid-2023.csv")

df = pd.concat([df2021Q1,df2021Q2,df2021Q3,df2021Q4,df2023], ignore_index=True) #df2019Q1,df2019Q2,df2019Q3,df2019Q4,df2020Q1,df2020Q2,df2020Q3,df2020Q4,

print(len(df.values))
print(len(df2023.values))
df.head(5)

5740102
3443752


,Date,Country,City,Specie,count,min,max,median,variance
0,2021-04-04,CN,Beijing,pressure,288,1022.0,1028.0,1026.0,48.39
1,2021-02-13,CN,Beijing,pressure,288,1018.0,1022.5,1021.0,23.31
2,2021-02-16,CN,Beijing,pressure,288,1018.0,1030.0,1026.0,150.71
3,2021-03-01,CN,Beijing,pressure,288,1026.5,1032.0,1028.5,25.09
4,2021-03-14,CN,Beijing,pressure,288,1010.5,1019.0,1015.5,93.17


Large scale data

Date, City, County,State,Population Staying at Home,Population Not Staying at Home,mil_miles,past_week_avg_miles,latitude,longitude,

o3_min,o3_max,o3_median,o3_variance,o3_count, ##
pressure_min,pressure_max,pressure_median,pressure_variance,pressure_count,##
pm25_min,pm25_max,pm25_median,pm25_variance,pm25_count, ##
humidity_min,humidity_max,humidity_median,humidity_variance,humidity_count, ##
temperature_min,temperature_max,temperature_median,temperature_variance,temperature_count, ##
dew_min,dew_max,dew_median,dew_variance,dew_count, ##
no2_min,no2_max,no2_median,no2_variance,no2_count, ##
wind-speed_min,wind-speed_max,wind-speed_median,wind-speed_variance,wind-speed_count, ##
co_min,co_max,co_median,co_variance,co_count, ##
so2_min,so2_max,so2_median,so2_variance,so2_count, ##
pm10_min,pm10_max,pm10_median,pm10_variance,pm10_count, ##
wind-gust_min,wind-gust_max,wind-gust_median,wind-gust_variance,wind-gust_count, ##

pp_feat

In [4]:
# columns
print(set(df['Specie'].values))

{'temperature', 'mepaqi', 'pm1', 'aqi', 'dew', 'co', 'o3', 'pm25', 'wd', 'humidity', 'precipitation', 'pm10', 'wind-gust', 'neph', 'pressure', 'wind-speed', 'so2', 'uvi', 'no2'}


In [5]:
# Filtering the Country 

rows_to_delete = df[df['Country'] != Country].index

# Delete the rows
df = df.drop(rows_to_delete)
df.head(10)

print(len(df.values))

143650


In [6]:
# df.to_csv('./data/{}2021to2023.csv'.format(Country), index=False)

In [7]:
df['min'].values[0]

1012.0

In [8]:
min_humidity = []
max_humidity = []
median_humidity = []
variance_humidity = []

min_no2 = []
max_no2 = []
median_no2 = []
variance_no2 = []

min_precipitation = []
max_precipitation = []
median_precipitation = []
variance_precipitation = []

min_co = []
max_co = []
median_co = []
variance_co = []

min_dew = []
max_dew = []
median_dew = []
variance_dew = []

min_temperature = []
max_temperature = []
median_temperature = []
variance_temperature = []

min_pm25 = []
max_pm25 = []
median_pm25 = []
variance_pm25 = []

min_wind_gust = []
max_wind_gust = []
median_wind_gust = []
variance_wind_gust = []

min_pm10 = []
max_pm10 = []
median_pm10 = []
variance_pm10 = []

min_o3 = []
max_o3 = []
median_o3 = []
variance_o3 = []

min_wd = []
max_wd = []
median_wd = []
variance_wd = []

min_pressure = []
max_pressure = []
median_pressure = []
variance_pressure = []

min_so2 = []
max_so2 = []
median_so2 = []
variance_so2 = []

min_wind_speed = []
max_wind_speed = []
median_wind_speed = []
variance_wind_speed = []        

'humidity', 'no2', 'precipitation', 'co', 'dew', 'temperature', 'pm25', 'wind-gust', 'pm10', 'o3', 'wd', 'pressure', 'so2', 'wind-speed'

In [9]:
def append_9999(lists):
    for list in lists:
        list.append(-9999)

In [10]:
print(len(df.values))

for num_row in range(len(df.values)):
    component = df['Specie'].values[num_row]
    min = df['min'].values[num_row]
    max = df['max'].values[num_row]
    median = df['median'].values[num_row]
    variance = df['variance'].values[num_row]
    if component == 'so2':
        min_so2.append(min)
        max_so2.append(max)
        median_so2.append(median)
        variance_so2.append(variance)
        other_components = [min_humidity, min_no2, min_precipitation, min_co, min_dew, min_temperature, min_pm25, min_wind_gust, min_pm10, min_o3, min_wd, min_pressure, min_wind_speed, 
                            max_humidity, max_no2, max_precipitation, max_co, max_dew, max_temperature, max_pm25, max_wind_gust, max_pm10, max_o3, max_wd, max_pressure, max_wind_speed, 
                            median_humidity, median_no2, median_precipitation, median_co, median_dew, median_temperature, median_pm25, median_wind_gust, median_pm10, median_o3, median_wd, median_pressure, median_wind_speed,
                            variance_humidity, variance_no2, variance_precipitation, variance_co, variance_dew, variance_temperature, variance_pm25, variance_wind_gust, variance_pm10, variance_o3, variance_wd, variance_pressure, variance_wind_speed]
        append_9999(other_components) 
    elif component == 'humidity':
        min_humidity.append(min)
        max_humidity.append(max)
        median_humidity.append(median)
        variance_humidity.append(variance)
        other_components = [min_no2, min_precipitation, min_co, min_dew, min_temperature, min_pm25, min_wind_gust, min_pm10, min_o3, min_wd, min_pressure, min_so2, min_wind_speed,
                            max_no2, max_precipitation, max_co, max_dew, max_temperature, max_pm25, max_wind_gust, max_pm10, max_o3, max_wd, max_pressure, max_so2, max_wind_speed, 
                            median_no2, median_precipitation, median_co, median_dew, median_temperature, median_pm25, median_wind_gust, median_pm10, median_o3, median_wd, median_pressure, median_so2, median_wind_speed, 
                            variance_no2, variance_precipitation, variance_co, variance_dew, variance_temperature, variance_pm25, variance_wind_gust, variance_pm10, variance_o3, variance_wd, variance_pressure, variance_so2, variance_wind_speed]
        append_9999(other_components)
    elif component == 'no2':
        min_no2.append(min)
        max_no2.append(max)
        median_no2.append(median)
        variance_no2.append(variance)
        other_components = [min_humidity, min_precipitation, min_co, min_dew, min_temperature, min_pm25, min_wind_gust, min_pm10, min_o3, min_wd, min_pressure, min_so2, min_wind_speed, 
                            max_humidity, max_precipitation, max_co, max_dew, max_temperature, max_pm25, max_wind_gust, max_pm10, max_o3, max_wd, max_pressure, max_so2, max_wind_speed, 
                            median_humidity, median_precipitation, median_co, median_dew, median_temperature, median_pm25, median_wind_gust, median_pm10, median_o3, median_wd, median_pressure, median_so2, median_wind_speed,
                            variance_humidity, variance_precipitation, variance_co, variance_dew, variance_temperature, variance_pm25, variance_wind_gust, variance_pm10, variance_o3, variance_wd, variance_pressure, variance_so2, variance_wind_speed]
        append_9999(other_components)
    elif component == 'precipitation':
        min_precipitation.append(min)
        max_precipitation.append(max)
        median_precipitation.append(median)
        variance_precipitation.append(variance)
        other_components = [min_humidity, min_no2, min_co, min_dew, min_temperature, min_pm25, min_wind_gust, min_pm10, min_o3, min_wd, min_pressure, min_so2, min_wind_speed, 
                            max_humidity, max_no2, max_co, max_dew, max_temperature, max_pm25, max_wind_gust, max_pm10, max_o3, max_wd, max_pressure, max_so2, max_wind_speed, 
                            median_humidity, median_no2, median_co, median_dew, median_temperature, median_pm25, median_wind_gust, median_pm10, median_o3, median_wd, median_pressure, median_so2, median_wind_speed,
                            variance_humidity, variance_no2, variance_co, variance_dew, variance_temperature, variance_pm25, variance_wind_gust, variance_pm10, variance_o3, variance_wd, variance_pressure, variance_so2, variance_wind_speed]
        append_9999(other_components)
    elif component == 'co':
        min_co.append(min)
        max_co.append(max)
        median_co.append(median)
        variance_co.append(variance)
        other_components = [min_humidity, min_no2, min_precipitation, min_dew, min_temperature, min_pm25, min_wind_gust, min_pm10, min_o3, min_wd, min_pressure, min_so2, min_wind_speed, 
                            max_humidity, max_no2, max_precipitation, max_dew, max_temperature, max_pm25, max_wind_gust, max_pm10, max_o3, max_wd, max_pressure, max_so2, max_wind_speed, 
                            median_humidity, median_no2, median_precipitation, median_dew, median_temperature, median_pm25, median_wind_gust, median_pm10, median_o3, median_wd, median_pressure, median_so2, median_wind_speed,
                            variance_humidity, variance_no2, variance_precipitation, variance_dew, variance_temperature, variance_pm25, variance_wind_gust, variance_pm10, variance_o3, variance_wd, variance_pressure, variance_so2, variance_wind_speed]
        append_9999(other_components)
    elif component == 'dew':
        min_dew.append(min)
        max_dew.append(max)
        median_dew.append(median)
        variance_dew.append(variance)
        other_components = [min_humidity, min_no2, min_precipitation, min_co, min_temperature, min_pm25, min_wind_gust, min_pm10, min_o3, min_wd, min_pressure, min_so2, min_wind_speed, 
                            max_humidity, max_no2, max_precipitation, max_co, max_temperature, max_pm25, max_wind_gust, max_pm10, max_o3, max_wd, max_pressure, max_so2, max_wind_speed, 
                            median_humidity, median_no2, median_precipitation, median_co, median_temperature, median_pm25, median_wind_gust, median_pm10, median_o3, median_wd, median_pressure, median_so2, median_wind_speed,
                            variance_humidity, variance_no2, variance_precipitation, variance_co, variance_temperature, variance_pm25, variance_wind_gust, variance_pm10, variance_o3, variance_wd, variance_pressure, variance_so2, variance_wind_speed]
        append_9999(other_components)
    elif component == 'temperature':
        min_temperature.append(min)
        max_temperature.append(max)
        median_temperature.append(median)
        variance_temperature.append(variance)
        other_components = [min_humidity, min_no2, min_precipitation, min_co, min_dew, min_pm25, min_wind_gust, min_pm10, min_o3, min_wd, min_pressure, min_so2, min_wind_speed, 
                            max_humidity, max_no2, max_precipitation, max_co, max_dew, max_pm25, max_wind_gust, max_pm10, max_o3, max_wd, max_pressure, max_so2, max_wind_speed, 
                            median_humidity, median_no2, median_precipitation, median_co, median_dew, median_pm25, median_wind_gust, median_pm10, median_o3, median_wd, median_pressure, median_so2, median_wind_speed,
                            variance_humidity, variance_no2, variance_precipitation, variance_co, variance_dew, variance_pm25, variance_wind_gust, variance_pm10, variance_o3, variance_wd, variance_pressure, variance_so2, variance_wind_speed]
        append_9999(other_components)
    elif component == 'pm25':
        min_pm25.append(min)
        max_pm25.append(max)
        median_pm25.append(median)
        variance_pm25.append(variance)
        other_components = [min_humidity, min_no2, min_precipitation, min_co, min_dew, min_temperature, min_wind_gust, min_pm10, min_o3, min_wd, min_pressure, min_so2, min_wind_speed, 
                            max_humidity, max_no2, max_precipitation, max_co, max_dew, max_temperature, max_wind_gust, max_pm10, max_o3, max_wd, max_pressure, max_so2, max_wind_speed, 
                            median_humidity, median_no2, median_precipitation, median_co, median_dew, median_temperature, median_wind_gust, median_pm10, median_o3, median_wd, median_pressure, median_so2, median_wind_speed,
                            variance_humidity, variance_no2, variance_precipitation, variance_co, variance_dew, variance_temperature, variance_wind_gust, variance_pm10, variance_o3, variance_wd, variance_pressure, variance_so2, variance_wind_speed]
        append_9999(other_components)
    elif component == 'wind-gust':
        min_wind_gust.append(min)
        max_wind_gust.append(max)
        median_wind_gust.append(median)
        variance_wind_gust.append(variance)
        other_components = [min_humidity, min_no2, min_precipitation, min_co, min_dew, min_temperature, min_pm25, min_pm10, min_o3, min_wd, min_pressure, min_so2, min_wind_speed, 
                            max_humidity, max_no2, max_precipitation, max_co, max_dew, max_temperature, max_pm25, max_pm10, max_o3, max_wd, max_pressure, max_so2, max_wind_speed, 
                            median_humidity, median_no2, median_precipitation, median_co, median_dew, median_temperature, median_pm25, median_pm10, median_o3, median_wd, median_pressure, median_so2, median_wind_speed,
                            variance_humidity, variance_no2, variance_precipitation, variance_co, variance_dew, variance_temperature, variance_pm25, variance_pm10, variance_o3, variance_wd, variance_pressure, variance_so2, variance_wind_speed]
        append_9999(other_components)
    elif component == 'pm10':
        min_pm10.append(min)
        max_pm10.append(max)
        median_pm10.append(median)
        variance_pm10.append(variance)
        other_components = [min_humidity, min_no2, min_precipitation, min_co, min_dew, min_temperature, min_pm25, min_wind_gust, min_o3, min_wd, min_pressure, min_so2, min_wind_speed, 
                            max_humidity, max_no2, max_precipitation, max_co, max_dew, max_temperature, max_pm25, max_wind_gust, max_o3, max_wd, max_pressure, max_so2, max_wind_speed, 
                            median_humidity, median_no2, median_precipitation, median_co, median_dew, median_temperature, median_pm25, median_wind_gust, median_o3, median_wd, median_pressure, median_so2, median_wind_speed,
                            variance_humidity, variance_no2, variance_precipitation, variance_co, variance_dew, variance_temperature, variance_pm25, variance_wind_gust, variance_o3, variance_wd, variance_pressure, variance_so2, variance_wind_speed]
        append_9999(other_components)
    elif component == 'o3':
        min_o3.append(min)
        max_o3.append(max)
        median_o3.append(median)
        variance_o3.append(variance)
        other_components = [min_humidity, min_no2, min_precipitation, min_co, min_dew, min_temperature, min_pm25, min_wind_gust, min_pm10, min_wd, min_pressure, min_so2, min_wind_speed, 
                            max_humidity, max_no2, max_precipitation, max_co, max_dew, max_temperature, max_pm25, max_wind_gust, max_pm10, max_wd, max_pressure, max_so2, max_wind_speed, 
                            median_humidity, median_no2, median_precipitation, median_co, median_dew, median_temperature, median_pm25, median_wind_gust, median_pm10, median_wd, median_pressure, median_so2, median_wind_speed,
                            variance_humidity, variance_no2, variance_precipitation, variance_co, variance_dew, variance_temperature, variance_pm25, variance_wind_gust, variance_pm10, variance_wd, variance_pressure, variance_so2, variance_wind_speed]
        append_9999(other_components)
    elif component == 'wd':
        min_wd.append(min)
        max_wd.append(max)
        median_wd.append(median)
        variance_wd.append(variance)
        other_components = [min_humidity, min_no2, min_precipitation, min_co, min_dew, min_temperature, min_pm25, min_wind_gust, min_pm10, min_o3, min_pressure, min_so2, min_wind_speed, 
                            max_humidity, max_no2, max_precipitation, max_co, max_dew, max_temperature, max_pm25, max_wind_gust, max_pm10, max_o3, max_pressure, max_so2, max_wind_speed, 
                            median_humidity, median_no2, median_precipitation, median_co, median_dew, median_temperature, median_pm25, median_wind_gust, median_pm10, median_o3, median_pressure, median_so2, median_wind_speed,
                            variance_humidity, variance_no2, variance_precipitation, variance_co, variance_dew, variance_temperature, variance_pm25, variance_wind_gust, variance_pm10, variance_o3, variance_pressure, variance_so2, variance_wind_speed]
        append_9999(other_components)
    elif component == 'pressure':
        min_pressure.append(min)
        max_pressure.append(max)
        median_pressure.append(median)
        variance_pressure.append(variance)
        other_components = [min_humidity, min_no2, min_precipitation, min_co, min_dew, min_temperature, min_pm25, min_wind_gust, min_pm10, min_o3, min_wd, min_so2, min_wind_speed, 
                            max_humidity, max_no2, max_precipitation, max_co, max_dew, max_temperature, max_pm25, max_wind_gust, max_pm10, max_o3, max_wd, max_so2, max_wind_speed, 
                            median_humidity, median_no2, median_precipitation, median_co, median_dew, median_temperature, median_pm25, median_wind_gust, median_pm10, median_o3, median_wd, median_so2, median_wind_speed,
                            variance_humidity, variance_no2, variance_precipitation, variance_co, variance_dew, variance_temperature, variance_pm25, variance_wind_gust, variance_pm10, variance_o3, variance_wd, variance_so2, variance_wind_speed]
        append_9999(other_components)
    elif component == 'wind-speed':
        min_wind_speed.append(min)
        max_wind_speed.append(max)
        median_wind_speed.append(median)
        variance_wind_speed.append(variance)
        other_components = [min_humidity, min_no2, min_precipitation, min_co, min_dew, min_temperature, min_pm25, min_wind_gust, min_pm10, min_o3, min_wd, min_pressure, min_so2, 
                            max_humidity, max_no2, max_precipitation, max_co, max_dew, max_temperature, max_pm25, max_wind_gust, max_pm10, max_o3, max_wd, max_pressure, max_so2, 
                            median_humidity, median_no2, median_precipitation, median_co, median_dew, median_temperature, median_pm25, median_wind_gust, median_pm10, median_o3, median_wd, median_pressure, median_so2,
                            variance_humidity, variance_no2, variance_precipitation, variance_co, variance_dew, variance_temperature, variance_pm25, variance_wind_gust, variance_pm10, variance_o3, variance_wd, variance_pressure, variance_so2]
        append_9999(other_components)   
    else:
        other_components = [min_humidity, min_no2, min_precipitation, min_co, min_dew, min_temperature, min_pm25, min_wind_gust, min_pm10, min_o3, min_wd, min_pressure, min_so2, min_wind_speed, 
                            max_humidity, max_no2, max_precipitation, max_co, max_dew, max_temperature, max_pm25, max_wind_gust, max_pm10, max_o3, max_wd, max_pressure, max_so2, max_wind_speed, 
                            median_humidity, median_no2, median_precipitation, median_co, median_dew, median_temperature, median_pm25, median_wind_gust, median_pm10, median_o3, median_wd, median_pressure, median_so2, median_wind_speed,
                            variance_humidity, variance_no2, variance_precipitation, variance_co, variance_dew, variance_temperature, variance_pm25, variance_wind_gust, variance_pm10, variance_o3, variance_wd, variance_pressure, variance_so2, variance_wind_speed]
        append_9999(other_components)
        
    
        

print(len(min_so2))
print(len(max_no2))
print(len(median_so2))
print(len(variance_so2))

143650
143650
143650
143650
143650


In [11]:
df['min_so2'] = min_so2
df['max_so2'] = max_so2
df['median_so2'] = median_so2
df['variance_so2'] = variance_so2
df['min_humidity'] = min_humidity
df['max_humidity'] = max_humidity
df['median_humidity'] = median_humidity
df['variance_humidity'] = variance_humidity
df['min_no2'] = min_no2
df['max_no2'] = max_no2
df['median_no2'] = median_no2
df['variance_no2'] = variance_no2
df['min_co'] = min_co
df['max_co'] = max_co
df['median_co'] = median_co
df['variance_co'] = variance_co
df['min_dew'] = min_dew
df['max_dew'] = max_dew
df['median_dew'] = median_dew
df['variance_dew'] = variance_dew
df['min_precipitation'] = min_precipitation
df['max_precipitation'] = max_precipitation
df['median_precipitation'] = median_precipitation
df['variance_precipitation'] = variance_precipitation
df['min_pm25'] = min_pm25
df['max_pm25'] = max_pm25
df['median_pm25'] = median_pm25
df['variance_pm25'] = variance_pm25
df['min_pm10'] = min_pm10
df['max_pm10'] = max_pm10
df['median_pm10'] = median_pm10
df['variance_pm10'] = variance_pm10
df['min_wind_gust'] = min_wind_gust
df['max_wind_gust'] = max_wind_gust
df['median_wind_gust'] = median_wind_gust
df['variance_wind_gust'] = variance_wind_gust
df['min_temperature'] = min_temperature
df['max_temperature'] = max_temperature
df['median_temperature'] = median_temperature
df['variance_temperature'] = variance_temperature
df['min_o3'] = min_o3
df['max_o3'] = max_o3
df['median_o3'] = median_o3
df['variance_o3'] = variance_o3
df['min_wd'] = min_wd
df['max_wd'] = max_wd
df['median_wd'] = median_wd
df['variance_wd'] = variance_wd
df['min_pressure'] = min_pressure
df['max_pressure'] = max_pressure
df['median_pressure'] = median_pressure
df['variance_pressure'] = variance_pressure
df['min_wind_speed'] = min_wind_speed
df['max_wind_speed'] = max_wind_speed
df['median_wind_speed'] = median_wind_speed
df['variance_wind_speed'] = variance_wind_speed

df.head(5)

,Date,Country,City,Specie,count,min,max,median,variance,min_so2,max_so2,median_so2,variance_so2,min_humidity,max_humidity,median_humidity,variance_humidity,min_no2,max_no2,median_no2,variance_no2,min_co,max_co,median_co,variance_co,min_dew,max_dew,median_dew,variance_dew,min_precipitation,max_precipitation,median_precipitation,variance_precipitation,min_pm25,max_pm25,median_pm25,variance_pm25,min_pm10,max_pm10,median_pm10,variance_pm10,min_wind_gust,max_wind_gust,median_wind_gust,variance_wind_gust,min_temperature,max_temperature,median_temperature,variance_temperature,min_o3,max_o3,median_o3,variance_o3,min_wd,max_wd,median_wd,variance_wd,min_pressure,max_pressure,median_pressure,variance_pressure,min_wind_speed,max_wind_speed,median_wind_speed,variance_wind_speed
59408,2021-03-24,IR,Isfahan,pressure,144,1012.0,1018.0,1015.5,42.94,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999,-9999,1012.0,1018.0,1015.5,42.94,-9999.0,-9999.0,-9999.0,-9999.0
59409,2021-03-26,IR,Isfahan,pressure,144,1015.0,1020.0,1017.0,19.02,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999,-9999,1015.0,1020.0,1017.0,19.02,-9999.0,-9999.0,-9999.0,-9999.0
59410,2021-01-18,IR,Isfahan,pressure,144,1014.0,1019.0,1016.0,21.71,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999,-9999,1014.0,1019.0,1016.0,21.71,-9999.0,-9999.0,-9999.0,-9999.0
59411,2021-01-22,IR,Isfahan,pressure,144,1024.5,1030.5,1028.0,29.37,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999,-9999,1024.5,1030.5,1028.0,29.37,-9999.0,-9999.0,-9999.0,-9999.0
59412,2021-01-23,IR,Isfahan,pressure,144,1026.0,1031.0,1028.5,32.71,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999,-9999,1026.0,1031.0,1028.5,32.71,-9999.0,-9999.0,-9999.0,-9999.0


In [12]:
# Aggregate the same row

aggregation_functions = {'Date': 'first', 'Country': 'first', 'City': 'first', 'min_humidity': 'max', 'min_no2': 'max', 'min_precipitation': 'max', 'min_co': 'max', 'min_dew': 'max', 'min_temperature': 'max', 'min_pm25': 'max', 'min_wind_gust': 'max', 'min_pm10': 'max', 'min_o3': 'max', 'min_wd': 'max', 'min_pressure': 'max', 'min_so2': 'max', 'min_wind_speed': 'max', 
                            'max_humidity': 'max', 'max_no2': 'max', 'max_precipitation': 'max', 'max_co': 'max', 'max_dew': 'max', 'max_temperature': 'max', 'max_pm25': 'max', 'max_wind_gust': 'max', 'max_pm10': 'max', 'max_o3': 'max', 'max_wd': 'max', 'max_pressure': 'max', 'max_so2': 'max', 'max_wind_speed': 'max', 
                            'median_humidity': 'max', 'median_no2': 'max', 'median_precipitation': 'max', 'median_co': 'max', 'median_dew': 'max', 'median_temperature': 'max', 'median_pm25': 'max', 'median_wind_gust': 'max', 'median_pm10': 'max', 'median_o3': 'max', 'median_wd': 'max', 'median_pressure': 'max', 'median_so2': 'max', 'median_wind_speed': 'max',
                            'variance_humidity': 'max', 'variance_no2': 'max', 'variance_precipitation': 'max', 'variance_co': 'max', 'variance_dew': 'max', 'variance_temperature': 'max', 'variance_pm25': 'max', 'variance_wind_gust': 'max', 'variance_pm10': 'max', 'variance_o3': 'max', 'variance_wd': 'max', 'variance_pressure': 'max', 'variance_so2': 'max', 'variance_wind_speed': 'max'}
df_grouped = df.groupby(['Date', 'City']).agg(aggregation_functions)
df_grouped.head(10)

Date Country          City  min_humidity  \
Date       City                                                           
2020-12-28 Arāk          2020-12-28      IR          Arāk          26.6   
           Isfahan       2020-12-28      IR       Isfahan          26.0   
           Karaj         2020-12-28      IR         Karaj          28.0   
           Kerman        2020-12-28      IR        Kerman          13.0   
           Kermanshah    2020-12-28      IR    Kermanshah          27.0   
           Khorramabad   2020-12-28      IR   Khorramabad          35.0   
           Khorramshahr  2020-12-28      IR  Khorramshahr          41.0   
           Mashhad       2020-12-28      IR       Mashhad          50.0   
           Orūmīyeh      2020-12-28      IR      Orūmīyeh          51.0   
           Qom           2020-12-28      IR           Qom          27.0   

                         min_no2  min_precipitation  min_co  min_dew  \
Date       City                                                        
2020-12-28 Arāk              5.0            -9999.0     7.0     -8.7   
           Isfahan       -9999.0            -9999.0 -9999.0    -12.0   
           Karaj            19.0            -9999.0     5.0     -9.0   
           Kerman           19.0            -9999.0     9.0    -17.0   
           Kermanshah       15.0            -9999.0     8.0     -7.0   
           Khorramabad      28.0            -9999.0     2.0      0.0   
           Khorramshahr  -9999.0            -9999.0    40.0      3.0   
           Mashhad       -9999.0            -9999.0     1.0     -6.5   
           Orūmīyeh         19.0            -9999.0 -9999.0    -10.0   
           Qom           -9999.0            -9999.0 -9999.0     -9.0   

                         min_temperature  min_pm25  min_wind_gust  min_pm10  \
Date       City                                                               
2020-12-28 Arāk                     -4.0      54.0        -9999.0      54.0   
           Isfahan                  -4.5      77.0        -9999.0   -9999.0   
           Karaj                    -5.0      78.0        -9999.0      54.0   
           Kerman                   -8.0      63.0        -9999.0   -9999.0   
           Kermanshah               -3.0      20.0        -9999.0      36.0   
           Khorramabad               4.0      19.0        -9999.0   -9999.0   
           Khorramshahr             10.0      64.0        -9999.0      52.0   
           Mashhad                  -5.0      36.0        -9999.0      12.0   
           Orūmīyeh                 -8.0   -9999.0        -9999.0      52.0   
           Qom                      -5.0      75.0        -9999.0      32.0   

                         min_o3  min_wd  min_pressure  min_so2  \
Date       City                                                  
2020-12-28 Arāk         -9999.0   -9999         833.7      9.0   
           Isfahan      -9999.0   -9999        1025.0  -9999.0   
           Karaj        -9999.0   -9999        1023.0      8.0   
           Kerman       -9999.0   -9999        1023.0  -9999.0   
           Kermanshah   -9999.0   -9999        1024.0  -9999.0   
           Khorramabad  -9999.0   -9999        1022.0     18.0   
           Khorramshahr -9999.0   -9999        1022.0     19.0   
           Mashhad      -9999.0   -9999        1026.0  -9999.0   
           Orūmīyeh     -9999.0   -9999        1026.0     20.0   
           Qom          -9999.0   -9999        1025.0     91.0   

                         min_wind_speed  max_humidity  max_no2  \
Date       City                                                  
2020-12-28 Arāk                     1.0          92.0    109.0   
           Isfahan                  0.5          79.0  -9999.0   
           Karaj                    1.0          85.0     44.0   
           Kerman                   1.0          67.0     23.0   
           Kermanshah               1.0          79.0    119.0   
           Khorramabad              1.0          75.0    107.

In [13]:
df_grouped.replace(-9999, np.nan, inplace=True)

df_grouped = df_grouped[~df_grouped['Date'].str.contains('2018-12-31',na=False,case=False)]
df_grouped = df_grouped[~df_grouped['Date'].str.contains('2018-12-30',na=False,case=False)]

# For removing August, 2023
for i in range(1,32):
    if i//10 == 0:
        df_grouped = df_grouped[~df_grouped['Date'].str.contains('2023-08-0{}'.format(i),na=False,case=False)]
        # print('2023-08-0{}'.format(i))
    else:
        df_grouped = df_grouped[~df_grouped['Date'].str.contains('2023-08-{}'.format(i),na=False,case=False)]
        # print('2023-08-{}'.format(i))

# df_grouped = df_grouped[~df_grouped['Date'].str.contains('2020-12-28',na=False,case=False)]
# df_grouped = df_grouped[~df_grouped['Date'].str.contains('2020-12-29',na=False,case=False)]
# df_grouped = df_grouped[~df_grouped['Date'].str.contains('2020-12-30',na=False,case=False)]
# df_grouped = df_grouped[~df_grouped['Date'].str.contains('2020-12-31',na=False,case=False)]


In [14]:
df_grouped.to_csv('./data/{}2021to2023.csv'.format(Country), index=False)

In [15]:
print(df_grouped['Country'].value_counts())
# print(df_grouped['Country'].nunique())
# print(df_grouped['City'].value_counts())
print(df_grouped['City'].nunique())
# print(df_grouped['Date'].value_counts())
print(df_grouped['Date'].nunique())


IR    15939
Name: Country, dtype: int64
18
918
